In [20]:
import pandas as pd
from sqlalchemy import create_engine, update, Table, MetaData
from dotenv import load_dotenv
import os, json, threading, time 
import google.generativeai as genai
from labels import get_prompt_A, get_prompt_main
from txt_to_df import txt_to_df
from test import get_test_df

load_dotenv(".env")
GEMINI_KEY = os.environ.get("GEMINI_KEY")
genai.configure(api_key=GEMINI_KEY)

# Gemini API # currently need VPN to outside Europe
model = genai.GenerativeModel('gemini-pro')

# SQLite for persistent storage
engine = create_engine('sqlite:///proposals.db')


In [5]:
pub_df = txt_to_df("tst.txt")
pub_df.shape

(484, 4)

In [23]:
publications = get_test_df()
publications["category"] = None
publications["reasoning"] = None
publications["subcategory"] = None
publications["subcategory_reasoning"] = None


# Write whole df to SQLite 
publications.to_sql('publications', con=engine, if_exists='replace', index=False)
publications_table = Table('publications', MetaData(), autoload_with=engine)
publications.shape

(18, 6)

In [25]:
def get_category(i):
 try:
    print(i)
    #get data from gemini api
    doc = publications["doc"][i]
    #title = publications["title"][i]
    #body = publications["body"][i]
    prompt = get_prompt_A("", doc)
    #print(prompt)
    res = model.generate_content(prompt)
    json_res = json.loads(res.text)#.replace("'", '"'))
    category = json_res["most_relevant_category"]
    reasoning = json_res["clear_reasoning"]
    print(i, category)
    #save in database
    stmt = (
    update(publications_table)
    .where(publications_table.c.doc == doc)
    .values(subcategory=category, subcategory_reasoning=reasoning)
    )
    with engine.connect() as conn:
     result = conn.execute(stmt)
     conn.commit()  
 except Exception as e:
   print(e)
   print(str(i)+"returned with err")

#test function
#get_category(1)


In [26]:
start_idx = 0
end_idx = 18

if __name__ == '__main__':
    threads = []
    for i in range(start_idx, end_idx):
        thread = threading.Thread(target=get_category, args=(i,))
        threads.append(thread)
        thread.start()
        time.sleep(1.01) # current rate limit 60 requests/minute
        if (i%10==0) :
            print("# running threads: " + str(threading.active_count()))

    for thread in threads:
        thread.join()


0
# running threads: 8
1
2
3
0 A4
4
1 A4
5
2 A9
4 A1
6
7
8
3 A6
5 A6
9
10
6 A5
# running threads: 11
11
7 A2
12
13
10 A4
14
11 A5
15
12 A5
9 A5
16
Expecting ',' delimiter: line 3 column 460 (char 494)
13returned with err
17
14 A1
15 A10
17 A7
16 A6
8 A5


In [27]:
query = "SELECT * FROM publications"
proposals = pd.read_sql_query(query, engine)

In [32]:
proposals.to_csv('new.csv', index=False, sep=";")

TypeError: "delimiter" must be a 1-character string

In [ ]:
# # #remove hallucinated labels
# labels = ["S", "OP", "TAM", "PRM", "PFU", "GAFM", "BAWM", "PED", "MISC"]
# # # Replace values not in the list with null
# proposals['category'] = proposals['category'].apply(lambda x: x if x in labels else None)
# proposals.to_sql('proposals', con=engine, if_exists='replace', index=False)

# filter for proposals that have not been assigned a label yet
proposals_no_labels = proposals[proposals['category'].isnull()]
proposals_no_labels = proposals_no_labels.reset_index()
print(proposals_no_labels.shape)
proposals_no_labels.head()